In [4]:
import pandas as pd
import xlrd
import math
import mip
import datetime

# costanti
N_TRATTE = 156
N_LUOGHI = 16

# lettura files
tratte = pd.read_excel('file_di_dati.xls', sheet_name="porzioni di viaggio")
tempi_vuoto = pd.read_excel('file_di_dati.xls', sheet_name="tempi di percorrenza a vuoto")


In [5]:
# funzione per convertie i tempi
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_sec(dt):
    return (dt - epoch).total_seconds()


nomi = {tempi_vuoto.iloc[0][i]:i for i in range(1,17)}

def t_trasf(i,j):
    a = nomi[tratte.iloc[j][2]]
    b = nomi[tratte.iloc[i][3]]
    #matrice riempita per metà...
    if(math.isnan(tempi_vuoto.iloc[a][b])):
        return datetime.timedelta(minutes=tempi_vuoto.iloc[b][a])
    return datetime.timedelta(minutes=tempi_vuoto.iloc[a][b])

Definiamo archi, nodi, flussi e costi

i nodi sono per ogni tratta 2
in particolare uno è la partenza e uno l' arrivo
definisco tutti i nodi, li collgo in base alla tratta 
definisco anche il nodo pozzo e il nodo sorgente

La matrice A ha tra (0,n-tratte*2) tutti i nodi 

In [6]:
#inizializzazione delle variabili
k = 33 #numero autobus
N_TRATTE = 156 #numero tratte
NUMERO_NODI = N_TRATTE * 2 + 2 #numero nodi del grafo
Archi = [[0]*NUMERO_NODI for i in range(NUMERO_NODI)]
costi = [[0]*NUMERO_NODI for i in range(NUMERO_NODI)]
flussi = [[0]*NUMERO_NODI for i in range(NUMERO_NODI)]
nodi = [0 for i in range(NUMERO_NODI)]
nodi[NUMERO_NODI-2] = (-1)*k #nodo radice
nodi[NUMERO_NODI-1] = k      #nodo pozzo

In [7]:
#matrice di compatibilità
for i in range(1, N_TRATTE+1):
    for j in range(i+1, N_TRATTE+1):
        t = t_trasf(i,j)
        if((tratte.iloc[i,5]+t)<=tratte.iloc[j,4]):
            Archi[i+N_TRATTE-1][j-1]=1
            flussi[i+N_TRATTE-1][j-1]=1 

In [8]:
#archi (i -> i') 
for i in range(N_TRATTE):
    for j in range(N_TRATTE):
        if(i==j):
            Archi[i][j+N_TRATTE]=1
            flussi[i][j+N_TRATTE]=1
            

#archi (r -> i)
for j in range(N_TRATTE):
    Archi[NUMERO_NODI-2][j]=1
    flussi[N_TRATTE-2][j]=1
    
#archi (i' -> p)
for i in range(N_TRATTE):
    Archi[i+N_TRATTE][NUMERO_NODI-1]=1
    flussi[i+N_TRATTE][NUMERO_NODI-1]=1
    
#arco (r -> p)
Archi[NUMERO_NODI-2][NUMERO_NODI-1]=1
flussi[NUMERO_NODI-2][NUMERO_NODI-1]=k  

In [9]:
#matrice dei costi
for i in range(N_TRATTE):
    for j in range(N_TRATTE):
        if(i==j):
            costi[i][j+N_TRATTE]=(-1)*(tratte.iloc[i+1][1])

In [10]:
# <<<<<< creazione modello MIP >>>>>>
m = mip.Model()
x = [m.add_var(var_type=mip.INTEGER) for h in range(NUMERO_NODI*NUMERO_NODI)]

In [11]:
#Vincoli di bilancio
for i in range(NUMERO_NODI):
    
    BS = []
    for k in range(NUMERO_NODI):
        if(Archi[k][i]>0):
            BS.append(k)
    FS = []
    for k in range(NUMERO_NODI):
        if(Archi[i][k]>0):
            FS.append(k)
            
    m.add_constr(  mip.xsum(x[(j*NUMERO_NODI)+i] for j in BS)  -  mip.xsum(x[(i*NUMERO_NODI)+j] for j in FS)  ==  nodi[i])

In [12]:
#Funzione obbiettivo
m.objective = mip.minimize(mip.xsum(costi[i][j]*x[(i*NUMERO_NODI)+j] for j in range(NUMERO_NODI) for i in range(NUMERO_NODI) if Archi[i][j]>0))

In [13]:
for i in range(NUMERO_NODI):
    for j in range(NUMERO_NODI): 
        if(Archi[i][j]>0):
            m.add_constr(x[(i*NUMERO_NODI)+j]>= 0)

In [14]:
for i in range(NUMERO_NODI):
    for j in range(NUMERO_NODI): 
        if(Archi[i][j]>0):
            m.add_constr(x[(i*NUMERO_NODI)+j]<= flussi[i][j])

In [15]:
#ottimizzazione
m.optimize()

#for i in range(dim):
    #print([int(x[(i*dim)+j].x) for j in range(dim)])

<OptimizationStatus.OPTIMAL: 0>

In [16]:
print('optimal solution cost: {} '.format(m.objective_value))

optimal solution cost: 0.0 
